In [27]:
%%capture
#!pip install python-constraint
# https://stackabuse.com/constraint-programming-with-python-constraint/
!git clone https://github.com/Ellariel/pv-opt

In [28]:
import json, time, requests, uuid, os, datetime, random
import pandas as pd, numpy as np, pickle
from tqdm.notebook import tqdm
from pandas import json_normalize
#import constraint, itertools
from datetime import date
from dateutil.rrule import rrule, DAILY

In [ ]:
d = pd.read_excel('/content/pv-opt/profiles.xls', sheet_name='G1', skiprows=2).dropna()
d.columns = ['t', 'winter_sam', 'winter_son', 'winter_werk',
                'sommer_sam', 'sommer_son', 'sommer_werk',
                'other_sam', 'other_son', 'other_werk']

months = {'January' : 'winter',
          'February' : 'winter',
          'December' : 'winter',
          'June' : 'sommer',
          'July' : 'sommer',
          'August' : 'sommer',
          'March' : 'other',
          'April' : 'other',
          'May' : 'other',
          'September' : 'other',
          'October' : 'other',
          'November' : 'other'}
a = date(2022, 1, 1)
b = date(2022, 12, 31)
consumption = None
for dt in rrule(DAILY, dtstart=a, until=b):
    season_key = months[dt.strftime("%B")]
    if dt.strftime('%A') == 'Saturday':
      day_key = 'sam'
    elif dt.strftime('%A') == 'Sunday':
      day_key = 'son'
    else:
      day_key = 'werk'
    s = d[['t', season_key+'_'+day_key]].copy()
    s.columns = ['t', 'w']
    #s['t'] = s['t'].apply(lambda x: datetime.datetime.strptime(dt.strftime("%Y-%m-%d") + ' ' + x.strftime("%H:%M:%S"), "%Y-%m-%d %H:%M:%S"))
    s['t'] = s['t'].apply(lambda x: dt.strftime("%Y-%m-%d") + ' ' + x.strftime("%H:%M:%S"))
    consumption = pd.concat([consumption, s], ignore_index=True)
#datetime.datetime.timestamp(
#consumption = {int(k): v['w'] for k, v in consumption.set_index('t').to_dict(orient='index').items()}
consumption['w'] = consumption['w']*10 #scale up to yearly consumption of 40.000 kWh
print(f"total W: {consumption['w'].sum()}")
consumption.to_csv('G1.csv', index=False)

total W: 41001442.0


In [34]:
d = pd.read_csv('/content/pv-opt/consumption/mook.csv')
d['w'] = d['w']/99+69000 #yearly consumption 600.000kWh
print(f"total W: {d['w'].sum()}")
d.to_csv('pumps.csv', index=False)

total W: 606100479.0899453


In [ ]:
with open('/content/pv-opt/consumption/mook.json', 'r') as fp:
   d = pd.read_json(json.load(fp)).reset_index()
   d.columns = ['t', 'w']
   print(f"total kW: {d['w'].sum()}")
   d.to_csv('mook.csv', index=False)

FileNotFoundError: ignored

In [ ]:
d

,t,w
0,2016-01-01 00:11:00,18.300818
1,2016-01-01 01:11:00,74.270985
2,2016-01-01 02:11:00,43.797255
3,2016-01-01 03:11:00,62.743968
4,2016-01-01 04:11:00,96.845059
...,...,...
8779,2016-12-31 19:11:00,83.215867
8780,2016-12-31 20:11:00,37.806819
8781,2016-12-31 21:11:00,13.780071
8782,2016-12-31 22:11:00,29.895713


In [ ]:
d = pd.read_excel('/content/pv-opt/profiles.xls', sheet_name='G1', skiprows=2).dropna()
d.columns = ['t', 'winter_sam', 'winter_son', 'winter_werk',
                'sommer_sam', 'sommer_son', 'sommer_werk',
                'other_sam', 'other_son', 'other_werk']
#d = d.set_index('t')
months = {'January' : 'winter',
          'February' : 'winter',
          'December' : 'winter',
          'June' : 'sommer',
          'July' : 'sommer',
          'August' : 'sommer',
          'March' : 'other',
          'April' : 'other',
          'May' : 'other',
          'September' : 'other',
          'October' : 'other',
          'November' : 'other'}

In [ ]:
a = date(2022, 1, 1)
b = date(2022, 12, 31)
consumption = None
for dt in rrule(DAILY, dtstart=a, until=b):
    season_key = months[dt.strftime("%B")]
    if dt.strftime('%A') == 'Saturday':
      day_key = 'sam'
    elif dt.strftime('%A') == 'Sunday':
      day_key = 'son'
    else:
      day_key = 'werk'
    s = d[['t', season_key+'_'+day_key]].copy()
    s.columns = ['t', 'w']
    s['t'] = s['t'].apply(lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(dt.strftime("%Y-%m-%d") + ' ' + x.strftime("%H:%M:%S"), "%Y-%m-%d %H:%M:%S")))
    consumption = pd.concat([consumption, s], ignore_index=True)
    #print(s.to_dict(orient='records'))
    #print(dt.strftime("%Y-%m-%d"))

In [ ]:
consumption = {k: v['w'] for k, v in consumption.set_index('t').to_dict(orient='index').items()}

In [ ]:
# https://docs.google.com/document/d/1hWvkCNNcFIAOSqk53JyHz27B5fMyvCu5adyz0-7fM44/edit

In [ ]:
cd pv-opt

/content/pv-opt


In [ ]:
from utils import Cache
from equip import Equipment, Location, Building, Battery, _calc_battery_costs

In [ ]:
# IC – installation costs / kWp and installation costs / kWh
# solar panel overproduction SPO = max{0, (building solar production - building consumption)}
# solar energy consumption SEC = min{building solar production, building consumption}
# solar panel underproduction SPU = max{0, (building consumption - building solar production)}
# grid buying price BP: price at which the respective energy provider is buying energy
# grid selling price SP: price at which the respective energy provider is selling energy
# city price CP: price at which the Genossenschaft is selling energy to the city # city_solar_energy_price
# max: Genossenschaft profit = (SPO * BP) + (SEC * CP) – IC - roof renting costs
# min: building energy cost = (SEC * CP) + (SPU * SP) + roof renting costs

def total_building_energy_costs(b, city_solar_energy_price=1, grid_selling_price=1.5):
  return b.total_solar_energy_consumption * city_solar_energy_price +\
         b.total_solar_energy_underproduction * grid_selling_price +\
         b.total_renting_costs

In [ ]:
bt1 = Battery()
bt2 = Battery()
bt2.battery_capacity_Ah = 60
bt2.battery_energy_Wh = 2880
bt2.battery_voltage = 48
bt2.battery_discharge_factor = 0.6
bt2.battery_price_per_Wh = 3.738

eq1 = Equipment()
eq2 = Equipment()
eq2.pv_size_mm = (eq2.pv_size_mm[0] * 3, eq2.pv_size_mm[1] * 3)

loc1 = Location()
loc1.equipment = [eq1]
loc2 = Location()
loc2.equipment = [eq2]

b1 = Building()
b1.locations = [loc1, loc2]
building = b1

print(f"total_production: {building.production['production'].sum():.1f}")
print(f"total_consumption: {building.consumption['consumption'].sum():.1f}")
print(f"total_solar_energy_consumption: {building.total_solar_energy_consumption:.1f}")
print(f"total_solar_energy_underproduction: {building.total_solar_energy_underproduction:.1f}")
print(f"total_renting_costs: {building.total_renting_costs:.1f}")

possible_locations = {
    1: loc1,
    2: loc2,
}
possible_equipment = {
    1: eq1,
    2: eq2,
}
possible_battery = {
    1: bt1,
    2: bt2,
}

locations_combinations = list(itertools.combinations(range(0, len(possible_locations)+1), len(possible_locations)))

def _locations(combination):
  return [possible_locations[i] for i in combination if i in possible_locations]

cached_solutions = Cache()

def get_cached_solution(building, A, B, C, D, E):
    def _calc():
        locations, eq, eq_count, bt, bt_count = _locations(A), possible_equipment[B], C, possible_battery[D], E
        eq.pv_count = eq_count
        locations[0].equipment = [eq]
        building.locations = locations
        building.updated()
        _total_energy_storage_needed = building.get_total_energy_storage_needed()
        _total_building_energy_costs = total_building_energy_costs(building)
        return _total_building_energy_costs, _total_energy_storage_needed
    return cached_solutions.get_cached_solution((A, B, C), _calc)

total_production: 799276.3
total_consumption: 886859.8
total_solar_energy_consumption: 799276.3
total_solar_energy_underproduction: 87583.5
total_renting_costs: 23.9


In [ ]:
locations_combinations

[(0, 1), (0, 2), (1, 2)]

In [ ]:
%%time
problem = constraint.Problem()
problem.addVariable('A', locations_combinations) # locations involved
problem.addVariable('B', range(1, len(possible_equipment)+1)) # equipment id
problem.addVariable('C', range(1, 7)) # equipment count
problem.addVariable('D', range(1, len(possible_battery)+1)) # battery id
problem.addVariable('E', range(0, 7)) # battery count

def equipment_square_constraint(A, B, C):
    locations, eq, eq_count = _locations(A), possible_equipment[B], C
    total_square_available = sum([loc.size_m[0] * loc.size_m[1] * 10 ** 6 for loc in locations])
    return eq.pv_size_mm[0] * eq.pv_size_mm[1] * eq_count < total_square_available

def battery_voltage_constraint(B, D):
    eq, bt = possible_equipment[B], possible_battery[D]
    return eq.pv_voltage == bt.battery_voltage

def battery_capacity_constraint(A, B, C, D, E):
    bt, bt_count = possible_battery[D], E
    _, _total_energy_storage_needed = get_cached_solution(building, A, B, C, D, E)
    return (bt.battery_energy_Wh * bt.battery_discharge_factor * bt_count >= _total_energy_storage_needed)

problem.addConstraint(equipment_square_constraint, "ABC")
problem.addConstraint(battery_voltage_constraint, "BD")
problem.addConstraint(battery_capacity_constraint, "ABCDE")
solutions = problem.getSolutions()
cached_solutions.save()
print(f"solutions: {len(solutions)}, cahed: {len(cached_solutions.storage)}")

solutions: 310, cahed: 77
CPU times: user 4.13 ms, sys: 969 µs, total: 5.09 ms
Wall time: 5.04 ms


In [ ]:
%%time
optimal_costs = +np.Inf
optimal_solution = {}

def _calc_costs(solution):
    A, B, C, D, E = solution['A'], solution['B'], solution['C'], solution['D'], solution['E']
    _total_building_energy_costs, _total_energy_storage_needed = get_cached_solution(building, A, B, C, D, E)
    locations, eq, eq_count, bt, bt_count = _locations(A), possible_equipment[B], C, possible_battery[D], E
    bt.battery_count = bt_count
    return _total_building_energy_costs + _calc_battery_costs(bt)

sorted_solutions = sorted(solutions, key=_calc_costs)
optimal_solution = sorted_solutions[0]

print(f'top-3 solutions:')
for i in sorted_solutions[:3]:
    print(i, 'cost:', _calc_costs(i))

print()
print(f"optimal solution: {optimal_solution}")
A, B, C, D, E = optimal_solution['A'], optimal_solution['B'], optimal_solution['C'], optimal_solution['D'], optimal_solution['E']
locations, eq, eq_count, bt, bt_count = _locations(A), possible_equipment[B], C, possible_battery[D], E
bt.battery_count = bt_count
eq.battery = [bt]
eq.pv_count = eq_count
locations[0].equipment = [eq]
building.locations = locations
building.updated()

print(f"total_production: {building.production['production'].sum():.1f}")
print(f"total_solar_energy_underproduction: {building.total_solar_energy_underproduction:.1f}")
print(f"total_building_energy_costs: {total_building_energy_costs(building):.1f}")
print(f"total_renting_costs: {building.total_renting_costs:.1f}")
print(f"total_battery_costs: {eq.total_battery_costs:.1f}")

top-3 solutions:
{'B': 1, 'D': 2, 'A': (0, 2), 'C': 3, 'E': 2} cost: 908397.8569226706
{'B': 1, 'D': 2, 'A': (0, 1), 'C': 3, 'E': 2} cost: 908397.8569226706
{'B': 1, 'D': 2, 'A': (0, 2), 'C': 4, 'E': 2} cost: 908400.2461706706

optimal solution: {'B': 1, 'D': 2, 'A': (0, 2), 'C': 3, 'E': 2}
total_production: 1198914.5
total_solar_energy_underproduction: 0.0
total_building_energy_costs: 886867.0
total_renting_costs: 7.2
total_battery_costs: 21530.9
CPU times: user 130 ms, sys: 4.71 ms, total: 135 ms
Wall time: 149 ms


In [ ]:
top-3 solutions:
{'B': 1, 'D': 2, 'A': (0, 2), 'C': 3, 'E': 2} cost: 908397.8569226706
{'B': 1, 'D': 2, 'A': (0, 1), 'C': 3, 'E': 2} cost: 908397.8569226706
{'B': 1, 'D': 2, 'A': (0, 2), 'C': 4, 'E': 2} cost: 908400.2461706706

optimal solution: {'B': 1, 'D': 2, 'A': (0, 2), 'C': 3, 'E': 2}
total_production: 1198914.5
total_solar_energy_underproduction: 0.0
total_building_energy_costs: 886867.0
total_renting_costs: 7.2
total_battery_costs: 21530.9
CPU times: user 98.4 ms, sys: 2.01 ms, total: 100 ms
Wall time: 105 ms


In [ ]:
total_energy_storage = peak_daily_consumption * autonomy_period_days
battery_capacity = total_energy_storage / battery_discharge_factor

* Determine Energy Requirements: Calculate the daily energy consumption of your appliances and devices that will be powered by the solar panel system. This will give you an estimate of the daily energy demand that the battery needs to fulfill.
* Account for Peak Load and Autonomy: Consider peak load times when your energy consumption might be higher than usual. Determine the desired autonomy period (number of days the battery can power your system without solar input) during cloudy or low-sunlight periods.
* Calculate Total Energy Storage Needed: Multiply the daily energy consumption by the desired autonomy period to calculate the total energy storage required. This will help you size the battery capacity.
* Choose Battery Type: Select the appropriate battery chemistry based on your requirements:
** Lead-Acid Batteries (Flooded, AGM, Gel): Affordable but lower cycle life and efficiency.
** Lithium-Ion Batteries (LiFePO4, NMC, LFP, etc.): Higher cycle life, efficiency, and energy density but generally costlier.
* Determine Battery Capacity: Divide the total energy storage needed by the battery's depth of discharge (DoD) factor. The DoD represents the portion of the battery's capacity that can be used without significantly affecting its lifespan. For example, if the battery has a DoD of 80%, then you can use 80% of its capacity.
* Consider System Voltage: Determine the system voltage based on your solar panel configuration (e.g., 12V, 24V, 48V). Choose a battery bank that matches the system voltage.

In [ ]:
https://www.youtube.com/watch?v=OPNBWaBZvjc&ab_channel=%D0%94%D0%B5%D1%80%D0%B5%D0%B2%D0%B5%D0%BD%D1%81%D0%BA%D0%B8%D0%B9%D1%84%D0%BE%D1%82%D0%BE%D0%B3%D1%80%D0%B0%D1%84

потребление - 1 кВт 8 часов в сутки
аккумулятор - 12 В х 60 Ah = 720 Втч * 0,7 = 504 Втч - полчас